In [39]:
import plotly.graph_objs as go
import pandas as pd
import requests
import plotly.plotly as py

In [2]:
reports_r = requests.get('https://peridash.ml/api/reports')
measurements_r = requests.get('https://peridash.ml/api/measurements')
epsas_r = requests.get('https://peridash.ml/api/epsas')
variables_r = requests.get('https://peridash.ml/api/variables')
indicators_r = requests.get('https://peridash.ml/api/indicators')

In [3]:
reports_df = pd.read_json(reports_r.text)
measurements_df = pd.read_json(measurements_r.text)
epsas_df = pd.read_json(epsas_r.text)
variables_df = pd.read_json(variables_r.text)
indicators_df = pd.read_json(indicators_r.text)

complete_reports_df = pd.merge(reports_df, epsas_df, left_on='epsa', right_on='url')
complete_measurements_df = pd.merge(measurements_df, epsas_df, left_on='epsa', right_on='url')

In [4]:
from plotly.offline import iplot, init_notebook_mode
init_notebook_mode(connected=True)

In [5]:
ind_names = [
    'Rendimiento actual de la fuente', 'Uso eficiente del recurso',
    'Cobertura de muestras de agua potable',
    'Conformidad de los análisis de agua potable realizados',
    'Dotación', 'Continuidad por racionamiento', 'Continuidad por corte',
    'Cobertura del servicio de agua potable',
    'Cobertura del servicio de alcantarillado sanitario',
    'Cobertura de micromedición',
    'Incidencia extracción de agua cruda subterránea ',
    'Índice de tratamiento de agua residual', 'Control de agua residual',
    'Capacidad instalada de planta de tratamiento de agua potable',
    'Capacidad instalada de planta de tratamiento de agua residual ',
    'Presión del servicio de agua potable',
    'Índice de agua no contabilizada en producción',
    'Índice de agua no contabilizada en la red',
    'Densidad de fallas en tuberías de agua potable',
    'Densidad de fallas en conexiones de agua potable',
    'Densidad de fallas en tuberías de agua residual',
    'Densidad de fallas en conexiones de agua residual',
    'Índice de operación eficiente', 'Prueba ácida',
    'Eficiencia de recaudación', 'Índice de endeudamiento total', 'Tarifa media',
    'Costo unitario de operación', 'Índice de ejecución de inversiones',
    'Personal calificado', 'Número de empleados por cada 1000 conexiones',
    'Atención de reclamos'
]
ind_units = ['%', '%', '%', '%', 'l/hab/día', 'hr/día', '%', '%', '%', '%',
             '%', '%', '%', '%', '%', '%', '%', '%', 'fallas/100km',
             'fallas/1000conex.', 'fallas/100km', 'fallas/1000conex.', '%', '-',
             '%', '%', '%CUO(Bs.)', '%TM(Bs.)', '%', '%', 'empleados/1000conex.', '%']

colors = [
    '#1f77b4','#ff7f0e','#2ca02c',
    '#d62728', '#9467bd', '#8c564b',
    '#e377c2', '#7f7f7f', '#bcbd22','#17becf'
]

def get_ind_name(ind):
    return ind_names[int(ind[3:])-1]

def get_ind_unit(ind):
    return ind_units[int(ind[3:])-1]

In [10]:
crdf = complete_reports_df
cmdf = complete_measurements_df

categories = list(crdf.category.sort_values().unique()) # ['A', 'B', 'C', 'D']
years = list(crdf.year.sort_values().unique()) # [2014, 2015, 2016, 2017]

selected_inds = ['ind23','ind27', 'ind28', 'ind25']

default_cat = 'A'
default_ind = 'ind23'

In [68]:
data = []

for cat_i, category in enumerate(categories):
    for ind_i, indicator in enumerate(selected_inds):
        visible = True if cat_i == 0 and ind_i == 0 else False
        
        for year_i, year in enumerate(years):
        
            filtered_df = cmdf[(cmdf.category == category) & (cmdf.year == year)]
                    
            trace = go.Bar(
                x=filtered_df.code,
                y=filtered_df[indicator],
                name=str(year),
                text=get_ind_unit(indicator),
                opacity=0.8,
                marker=dict(color=colors[year_i]),
                visible=visible
            )
            
            data.append(trace)   

In [69]:
def get_visible_list(category, indicator):
    category_map = dict(A=0, B=1, C=2, D=3)
    indicator_map = dict(ind23=0, ind27=1, ind28=2, ind25=3)
    
    cat_offset = category_map[category]
    ind_offset = indicator_map[indicator]
    
    offset = (cat_offset * 4) + ind_offset
    
    base_array = [0 for i in range(16)]
    base_array[offset] = 1
    
    big_true = [True for i in range(4)]
    big_false = [False for i in range(4)]
    
    nested_array = [big_true if x == 1 else big_false for x in base_array]
    
    return [x for l in nested_array for x in l]

def make_x_axis(category):
    return dict(
        title=f'EPSAs Categoría {category}',
        titlefont=dict(
            family='Courier New, monospace',
            size=18,
            color='#7f7f7f'
        )
    )

def make_y_axis(ind_code):
    return dict(
        title=f'Unidad: {get_ind_unit(ind_code)}',
        titlefont=dict(
            family='Courier New, monospace',
            size=18,
            color='#7f7f7f'
        )
    )

def make_title(ind_code):
    ind_name = get_ind_name(ind_code)
    ind_unit = get_ind_unit(ind_code)
    ind_param = '(Parámetro óptimo: 65-75%)' if ind_code == 'ind23' else ''
    return f'Indicador {ind_code[3:]}: {ind_name} ({ind_unit}) {ind_param}'
    

def make_shapes(ind_code):
    
    special_shapes = [dict(
        type= 'rect',
        xref= 'paper',
        yref= 'y',
        x0= 0,
        y0= 65,
        x1=1,
        y1= 75,
        fillcolor= colors[0],
        opacity= 0.2,
        line= {'width': 0,}
    )]
    
    return special_shapes if ind_code == 'ind23' else []

    
drop_x_map = dict(
    ind23=0,
    ind27=0.25,
    ind28=0.5,
    ind25=0.75,
)

def get_label(ind, cat):
    name_list = ['IOE', 'TM', 'CUO', 'ER']
    ind_map = dict(ind23=0, ind27=1, ind28=2, ind25=3)
    return f'{name_list[ind_map[ind]]} - {cat}'

In [71]:
updatemenus = [dict(
    showactive=True,
    active=0,
    xanchor='left',
    yanchor='top',
    direction='up',
    x= drop_x_map[ind],
    y= -0.5,
    buttons = [dict(
        label=get_label(ind, cat),
        method='update', # modify both data and layout
        args = [
            {'visible': get_visible_list(cat, ind)}, # data modification
            # layout modification
            dict(
                title= make_title(ind),
                xaxis= make_x_axis(cat),
                yaxis= make_y_axis(ind),
                shapes= make_shapes(ind),
            ),
        ],
    ) for cat in categories]
) for ind in selected_inds]

In [72]:
layout = go.Layout(
    title= make_title(default_ind),
    xaxis= make_x_axis(default_cat),
    yaxis= make_y_axis(default_ind),
    shapes=make_shapes(default_ind),
    hovermode='closest',
    updatemenus = updatemenus,
)

In [73]:
fig = go.Figure(data=data, layout=layout)
py.iplot(fig, filename='economicos_2017')